In [0]:
import numpy as np
import pandas as pd

In [2]:
!wget http://files.grouplens.org/datasets/movielens/ml-latest-small.zip

--2019-01-14 10:16:22--  http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.34.235
Connecting to files.grouplens.org (files.grouplens.org)|128.101.34.235|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 978202 (955K) [application/zip]
Saving to: ‘ml-latest-small.zip.1’

ml-latest-small.zip 100%[===================>] 955.28K  4.07MB/s    in 0.2s    

2019-01-14 10:16:22 (4.07 MB/s) - ‘ml-latest-small.zip.1’ saved [978202/978202]



In [3]:
!unzip ml-latest-small.zip

Archive:  ml-latest-small.zip
replace ml-latest-small/links.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ml-latest-small/links.csv  
replace ml-latest-small/tags.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ml-latest-small/tags.csv  
replace ml-latest-small/ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ml-latest-small/ratings.csv  
replace ml-latest-small/README.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ml-latest-small/README.txt  
replace ml-latest-small/movies.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ml-latest-small/movies.csv  


In [4]:
ratings_data = pd.read_csv('ml-latest-small/ratings.csv')
movie_names = pd.read_csv("ml-latest-small/movies.csv")
movie_data = pd.merge(ratings_data, movie_names, on='movieId')
movie_names.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
user_movie_rating = movie_data.pivot_table(index='userId', columns='movieId', values='rating').fillna(0)
user_movie_rating.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
user_movie_rating_mat = user_movie_rating.as_matrix()
user_ratings_mean = np.mean(user_movie_rating_mat, axis = 1)
user_movie_rating_mat_normalized = user_movie_rating_mat - user_ratings_mean.reshape(-1, 1)

In [0]:
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(user_movie_rating_mat_normalized, k = 50)
sigma = np.diag(sigma)

In [8]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)
preds_df = pd.DataFrame(all_user_predicted_ratings, columns = user_movie_rating.columns)
preds_df.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
0,2.167328,0.402751,0.840184,-0.076281,-0.551337,2.504091,-0.890114,-0.026443,0.196974,1.593259,...,-0.023453,-0.019967,-0.026939,-0.026939,-0.023453,-0.026939,-0.023453,-0.023453,-0.023453,-0.058732
1,0.211459,0.006658,0.033455,0.017419,0.183430,-0.062473,0.083037,0.024158,0.049330,-0.152530,...,0.019498,0.016777,0.022219,0.022219,0.019498,0.022219,0.019498,0.019498,0.019498,0.032281
2,0.003588,0.030518,0.046393,0.008176,-0.006247,0.107328,-0.012416,0.003779,0.007297,-0.059362,...,0.005909,0.006209,0.005610,0.005610,0.005909,0.005610,0.005909,0.005909,0.005909,0.008004
3,2.051549,-0.387104,-0.252199,0.087562,0.130465,0.270210,0.477835,0.040313,0.025858,-0.017365,...,0.004836,0.004172,0.005500,0.005500,0.004836,0.005500,0.004836,0.004836,0.004836,-0.023311
4,1.344738,0.778511,0.065749,0.111744,0.273144,0.584426,0.254930,0.128788,-0.085541,1.023455,...,-0.008042,-0.007419,-0.008664,-0.008664,-0.008042,-0.008664,-0.008042,-0.008042,-0.008042,-0.010127


In [9]:
def recommend_movies(predictions_df, userID, movies_df, original_ratings_df, num_recommendations=5):
    
    # Get and sort the user's predictions
    user_row_number = userID - 1 # UserID starts at 1, not 0
    sorted_user_predictions = predictions_df.iloc[user_row_number].sort_values(ascending=False)
    
    # Get the user's data and merge in the movie information.
    user_data = original_ratings_df[original_ratings_df.userId == (userID)]
    user_full = (user_data.merge(movies_df, how = 'left', left_on = 'movieId', right_on = 'movieId').
                     sort_values(['rating'], ascending=False)
                 )

    print('User {0} has already rated {1} movies.'.format(userID, user_full.shape[0]))
    print('Recommending the highest {0} predicted ratings movies not already rated.'.format(num_recommendations))
    
    # Recommend the highest predicted rating movies that the user hasn't seen yet.
    recommendations = (movies_df[~movies_df['movieId'].isin(user_full['movieId'])].
         merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
               left_on = 'movieId',
               right_on = 'movieId').
         rename(columns = {user_row_number: 'Predictions'}).
         sort_values('Predictions', ascending = False).
                       iloc[:num_recommendations, :-1]
                      )

    return user_full, recommendations

User 500 has already rated 86 movies.
Recommending the highest 10 predicted ratings movies not already rated.


In [15]:
already_rated, predictions = recommend_movies(preds_df, 3, movie_names, ratings_data, 10)

User 3 has already rated 39 movies.
Recommending the highest 10 predicted ratings movies not already rated.


In [16]:
already_rated.head(10)

,userId,movieId,rating,timestamp,title,genres
28,3,5181,5.0,1306463718,Hangar 18 (1980),Action|Sci-Fi|Thriller
37,3,70946,5.0,1306463815,Troll 2 (1990),Fantasy|Horror
35,3,7991,5.0,1306463684,Death Race 2000 (1975),Action|Sci-Fi
5,3,849,5.0,1306463611,Escape from L.A. (1996),Action|Adventure|Sci-Fi|Thriller
33,3,6835,5.0,1306463670,Alien Contamination (1980),Action|Horror|Sci-Fi
21,3,2851,5.0,1306463925,Saturn 3 (1980),Adventure|Sci-Fi|Thriller
31,3,5919,5.0,1306463825,Android (1982),Sci-Fi
24,3,3703,5.0,1306463603,"Road Warrior, The (Mad Max 2) (1981)",Action|Adventure|Sci-Fi|Thriller
29,3,5746,5.0,1306463708,Galaxy of Terror (Quest) (1981),Action|Horror|Mystery|Sci-Fi
26,3,4518,5.0,1306463770,The Lair of the White Worm (1988),Comedy|Horror


In [17]:
predictions

,movieId,title,genres
893,1200,Aliens (1986),Action|Adventure|Horror|Sci-Fi
906,1214,Alien (1979),Horror|Sci-Fi
847,1127,"Abyss, The (1989)",Action|Adventure|Sci-Fi|Thriller
1884,2529,Planet of the Apes (1968),Action|Drama|Sci-Fi
2226,2985,RoboCop (1987),Action|Crime|Drama|Sci-Fi|Thriller
2612,3527,Predator (1987),Action|Sci-Fi|Thriller
948,1258,"Shining, The (1980)",Horror
253,293,Léon: The Professional (a.k.a. The Professiona...,Action|Crime|Drama|Thriller
849,1129,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller
1043,1374,Star Trek II: The Wrath of Khan (1982),Action|Adventure|Sci-Fi|Thriller
